# Linear Optimization 

## Stock Data

### Overview of the stock data

$p_{st}$ - price of stock $s$ at the closing of the day $t$ <br>
$r_{st} = {p_{t+1} \over p_t}$ - daily return on investment of stock $s$ <br>
$r_s = {{1 \over T} \sum_t^T {p_{s,t+1} \over p_{s,t}}}$ - average return, where $T$ - last day of the period <br>
$d_{st} = r_{st} - r_{s}$ - daily risk of a stock (deviation of a daily return from the average return)


### Getting the data from yahoo finance

In [6]:
import pandas as pd
import pandas_datareader as pdr

In [45]:
stock_symbols = ['AMD', 'QCOM', 'INTC', 'NVDA', 'UMC']
stocks = {}

for symbol in stock_symbols:
    stock = {}
    df = pdr.get_data_yahoo(symbol, start="2021-01-01")
    df["DailyReturn"] = df["Close"].pct_change() 
    avg_return = df["DailyReturn"].sum() / len(df)
    df["DailyRisk"] = df["DailyReturn"].apply(lambda x: x-avg_return)
    df.drop(index=df.index[0],axis=0, inplace=True)
    
    stock["return"] = df["DailyReturn"].tolist()
    stock["risk"] = df["DailyRisk"].tolist()
    stock["avg_return"] = avg_return
    stock["last_price"] = df["Close"].iloc[-1]
    stocks[symbol] = stock

In [46]:
import json

with open('portfolio.json', 'w') as fp:
    json.dump(stocks, fp)

## Stock Portfolio Optimization Model: Mean Absolute Deviation (MAD)

**Mean Absolute Deviation (MAD)** model (Konno & Yamazaki*) uses observed data over the stock prices and minimizes the risk while exceeding a minimum return on investement.

$R$ - minimum guaranteed return on investment<br><br>
$I$ - total investment<br><br>
$z_s$ -  number of shares purchased of stock $s$<br><br>
$x_s = {p_{s,T}*z_s \over I}$ - proportion of the total investment invested in stock $s$<br><br>
$y_{st} = {d_{st}*x_s \over N_{stocks}}$ - average daily risk wheighted by share of stock $s$<br>

Model: <br><br>
$min{1 \over T} \sum_t^T {y_{t}}$<br><br>

The difference between the daily return compared to the average over all of the stocks has to be within $y_t$:<br>
$\forall t, \sum_s d_{st}x_s \geq -y_t$<br>
$\forall t, \sum_s d_{st}x_s \leq y_t$<br>

Averege wheighted return should be at least $R$:<br>
$\sum_s r_sx_s \geq R$<br>

The sum of all stocks proportion should be equal to 1:<br>
$\sum_s x_s = 1$<br>

The number of shares is an integer number as a share of investment devided by the last price of the stock:<br>
$z_s = {x_s*I \over p_{s,T}}$<br>


Proportion of each stock is non-begative: <br>
$\forall s, x_s \geq 0$<br>

Average daily risk is non-negative:<br>
$\forall t, y_t \geq 0$<br>

In [67]:
from ortools.linear_solver import pywraplp

import json
with open('portfolio.json', 'r') as fp:
    portfolio = json.load(fp)
    


#### 1. Instantiate Google's LP Solver

In [69]:
solver = pywraplp.Solver.CreateSolver('SCIP')

# daily return as a target
R = 0.0

# total investment
I = 10000

# total number of days in the analyzed data-set
T = len(list(portfolio.values())[0]['return'])

#total number of stocks in the portfolio
N = len(portfolio.keys())

#### 2. Adding variables to the model

In [71]:
# x = {stock_name: share} -proportion of each stock in the portfolio
# z - number of shares to purchase
x = {}
z = {}

for stock in portfolio:
    x[stock] = solver.NumVar(0,1, 'stock_proportion_'+str(stock))
    z[stock] = solver.IntVar(0, solver.infinity(), 'number of shares_'+str(stock))
    
# y = {day_number: y} -average daily risk wheighted by stock share
y = {}
for t in range(T):
    y[t] = solver.NumVar(0, solver.infinity(), 'avg_risk_'+str(t))

    
print('Number of variables =', solver.NumVariables())    



Number of variables = 766


#### 3. Adding constraints to the model

**3.1.** The difference between the daily return compared to the average over all of the stocks has to be within $y_{st}$:<br>
$\forall t, \sum_s d_{st}x_s \geq -y_{st}$<br>
$\forall t, \sum_s d_{st}x_s \leq y_{st}$<br>

In [72]:
for t in range(T):
    solver.Add(sum(x[stock]*portfolio[stock]["risk"][t] for stock in x) / N >= -y[t])
    solver.Add(sum(x[stock]*portfolio[stock]["risk"][t] for stock in x) / N <= y[t])
    


**3.2.** Avrerage return wheighted on share should be at least $R$:<br>
$\sum_s r_sx_s \geq R$<br>

In [73]:
solver.Add((sum(x[stock]*portfolio[stock]["avg_return"] for stock in portfolio)/ N) >= R)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001DD26167DB0> >

**3.3.** The sum of all stocks proportion should be equal to 1:<br>
$\sum_s x_s = 1$<br>

In [74]:
solver.Add(sum(x[stock] for stock in portfolio) == 1)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001DD261D1500> >

**3.4.** Number of shares for each stock:<br>
$z_s = {x_s*I \over p_{s,T}}$

In [75]:
for stock in z:
    solver.Add((x[stock]*I/portfolio[stock]["last_price"]) == z[stock])

#### 4. Defining optimization function

Sum of daily y's for all days should be minimum


In [76]:
solver.Minimize(sum(y[t] for t in y) / len(y.keys()))

#### 5. Solution

In [79]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    total_investment = 0.0
    for stock in portfolio:
        print('x_'+str(stock)+' =', x[stock].solution_value())
        print('z_'+str(stock)+" =", z[stock].solution_value(), "stock purchase = ", z[stock].solution_value()*portfolio[stock]["last_price"])
        total_investment += z[stock].solution_value()*portfolio[stock]["last_price"]
    print("Total investment =", total_investment)

Solution:
Objective value = 0.0033390191676388272
x_AMD = 0.08974899597167965
z_AMD = 11.0 stock purchase =  897.4899597167969
x_QCOM = 0.1878240051269531
z_QCOM = 14.0 stock purchase =  1878.2400512695312
x_INTC = 0.4271279949188231
z_INTC = 111.0 stock purchase =  4271.279949188232
x_NVDA = 0.14549544525146485
z_NVDA = 9.0 stock purchase =  1454.9544525146484
x_UMC = 0.14980355873107937
z_UMC = 210.0 stock purchase =  1498.035020828247
Total investment = 9999.999433517456
